# Introduction to Linear Programming with Pyomo

Using examples (2.2.1 to 2.2.6) within *Operations Research: A Practical Introduction* book, this Jupyter Notebook introduces linear programming (LP). A strong mathematical method for optimizing a linear function with an objective under a set of linear constraints is called linear programming. It is extensively used in several disciplines, including economics, engineering, logistics, and operations research.

This notebook will use **Pyomo**, a flexible and reliable Python optimization modeling language, to specify and tackle these linear programming problems. In order to improve scalability and problem management, we will use the **Abstract Model** approach, which divides problem definition from data specification.

---

## What is Pyomo?

A Python library called Pyomo (Python Optimization Modeling Objects) offers a flexible and user-friendly method for defining optimization models. Linear, nonlinear, and integer programming are among the problem types it supports. Pyomo allows you to:

- Define decision variables  
- Formulate constraints  
- Specify the objective function  
- Solve models using multiple solvers (like CBC, Gurobi, or CPLEX)  

Users can express models symbolically using Pyomo's Abstract Model method, which provides real problem data at runtime. Pyomo is an effective tool for defining scalable and flexible optimization problems because of this abstraction.

---

## What is the Abstract Model in Pyomo?

Pyomo's **Abstract Model** makes it possible to distinguish precisely between an optimization problem's input data and its structure. When testing different data scenarios with the same model structure, this modeling approach is perfect. Important advantages include:

- Enhanced problem scalability  
- Better model maintainability  
- Cleaner separation of data and problem logic  


# Setting Up Pyomo and Abstract Model

## Key Pyomo Components

### 1. **Defining Variables**
Variables represent decision points in an optimization problem. In Pyomo, they can be defined using:

```python
from pyomo.environ import *

model.x = Var(within=NonNegativeReals)
```

This line defines `x` as a variable constrained to take only non-negative real values.

### 2. **Defining Parameters**
Parameters are constants whose values remain fixed during optimization. For example:

```python
model.coeff = Param()
```

Parameters can be indexed, allowing the definition of multiple parameter values.

### 3. **Defining Objectives**
The objective function specifies what we want to maximize or minimize. For example:

```python
model.obj = Objective(rule=lambda model: model.coeff * model.x, sense=maximize)
```

### 4. **Defining Constraints**
Constraints restrict the feasible region of solutions:

```python
model.constraint1 = Constraint(rule=lambda model: model.x <= 10)
```

### 5. **Solver Setup**

Pyomo supports several solvers, such as CBC and GLPK. To solve a model, we use:

```python
solver = SolverFactory('cbc')
result = solver.solve(model)
```


# Example Code Walkthrough

Here, we analyze an example using Pyomo with abstract model.

## Example 2.2.1: Wireless Router Production

### Problem Statement
A manufacturer produces two models of wireless routers (Model A and Model B). Each model requires different amounts of materials and labor, and the manufacturer wants to maximize profits while staying within the available resources.


### Model Formulation
1. **Decision Variables**:
   - Let \( $x_A$ \) = number of Model A routers produced.
   - Let \( $x_B$ \) = number of Model B routers produced.


2. **Objective Function**:
   - Maximize profit: \( $z$ = 22 $x_A$ + 28 $x_B$ \).


3. **Constraints**:
   - Materials constraint: \( 8 $x_A$ + 10 $x_B$ $\leq$ 3400 \).
   - Labor constraint: \( 2 $x_A$ + 3 $x_B$ $\leq$ 960 \).
   - Non-negativity: \( $x_A$ $ \geq$ 0 \), \( $x_B$ $\geq$ 0 \).


### Model Definition
```python
def build_abstract_model_221():
    """
    Builds an abstract Pyomo model for a linear programming problem,
    representing a production planning scenario with two products.

    This model structure is designed to be flexible and reusable with
    different data sets by using abstract components (like Param).

    Returns:
        model: An abstract Pyomo model object.
    """
    model = AbstractModel()  # Creates an abstract model object to define the problem structure

    # Decision variables representing the production quantity of each product
    model.xA = Var(within=NonNegativeReals)  # Quantity of product A to produce (must be non-negative)
    model.xB = Var(within=NonNegativeReals)  # Quantity of product B to produce (must be non-negative)

    # Parameters that will hold the objective function coefficients
    model.obj_coeff_A = Param()  # Profit coefficient for product A
    model.obj_coeff_B = Param()  # Profit coefficient for product B

    # Parameters for the materials constraint
    model.materials_coeff_A = Param()  # Material usage coefficient for product A
    model.materials_coeff_B = Param()  # Material usage coefficient for product B
    model.materials_rhs = Param()      # Total available materials (right-hand side of the constraint)

    # Parameters for the labor constraint
    model.labor_coeff_A = Param()  # Labor usage coefficient for product A
    model.labor_coeff_B = Param()  # Labor usage coefficient for product B
    model.labor_rhs = Param()      # Total available labor (right-hand side of the constraint)

    # Defines the objective function to maximize profit
    def obj_rule(model):
        """Calculates the total profit based on production quantities and profit coefficients."""
        return model.obj_coeff_A * model.xA + model.obj_coeff_B * model.xB  # Profit = (Profit per A * A quantity) + (Profit per B * B quantity)
    model.obj = Objective(rule=obj_rule, sense=maximize)  # Sets the objective to maximization

    # Defines the materials constraint
    def materials_rule(model):
        """Ensures that the total material usage is within the available limit."""
        return model.materials_coeff_A * model.xA + model.materials_coeff_B * model.xB <= model.materials_rhs  # Material usage <= Available materials
    model.materials = Constraint(rule=materials_rule)  # Creates the materials constraint

    # Defines the labor constraint
    def labor_rule(model):
        """Ensures that the total labor usage is within the available limit."""
        return model.labor_coeff_A * model.xA + model.labor_coeff_B * model.xB <= model.labor_rhs  # Labor usage <= Available labor
    model.labor = Constraint(rule=labor_rule)  # Creates the labor constraint

    return model  # Returns the built abstract model
```

### Data Instantiation
```python
# Data for Example 2.2.1, structured as a dictionary for Pyomo
data_221 = {
    None: {  # This outer 'None' key is common in Pyomo abstract models
        'obj_coeff_A': {None: 22}, 'obj_coeff_B': {None: 28},  # Objective function coefficients for xA and xB
        'materials_coeff_A': {None: 8}, 'materials_coeff_B': {None: 10}, 'materials_rhs': {None: 3400},  # Materials constraint coefficients and RHS
        'labor_coeff_A': {None: 2}, 'labor_coeff_B': {None: 3}, 'labor_rhs': {None: 960}  # Labor constraint coefficients and RHS
    }
}

# Creates a concrete model instance by loading data into the abstract model.
model_221 = build_abstract_model_221().create_instance(data_221)

# Solves the model instance and prints the results.
solve_model(model_221, "Example 2.2.1")
```

### Explanation
- **Variables (`xA`, `xB`)**: Represent production levels.
- **Objective function (`obj`)**: Defines the profit to be maximized.
- **Constraints (`materials`, `labor`)**: Restrict production based on resource limits.

## Solver Interpretation
The solver reads the abstract model, substitutes data values, and searches for the optimal variable assignments satisfying all constraints.

### Code Implementation

In [ ]:
# Importing Pyomo environment
from pyomo.environ import * # Imports all components from the Pyomo environment, like Var, Param, Objective, Constraint, etc.

# Function to solve and display results of an optimization model
def solve_model(model, example_name):
    print("\n" + "="*40) # Prints a separator line for better output formatting.
    print(f" Solving {example_name} ") # Prints the name of the example being solved.
    print("="*40) # Prints another separator line.
    solver = SolverFactory('cbc') # Creates a solver object using the CBC solver.
    result = solver.solve(model) # Solves the given Pyomo model using the CBC solver.
    print("\nOptimal Solution:") # Prints a header for the optimal solution.
    for v in model.component_objects(Var, active=True): # Iterates through all active variables in the model.
        for index in v: # Iterates through any indices of the variable (if it's indexed).
            print(f"  {v[index]} = {v[index].value}") # Prints the variable name and its optimal value.
    print("\nObjective Value:", model.obj()) # Prints the optimal value of the objective function.

# Function to build the abstract Pyomo model for Example 2.2.1
def build_abstract_model_221():
    model = AbstractModel() # Creates an AbstractModel object, which is a Pyomo model that is defined without specific data values.
    model.xA = Var(within=NonNegativeReals) # Defines a decision variable `xA` representing the number of Model A routers produced, constrained to be non-negative.
    model.xB = Var(within=NonNegativeReals) # Defines a decision variable `xB` representing the number of Model B routers produced, constrained to be non-negative.
    model.obj_coeff_A = Param() # Defines a parameter `obj_coeff_A` for the objective function coefficient of `xA`.
    model.obj_coeff_B = Param() # Defines a parameter `obj_coeff_B` for the objective function coefficient of `xB`.
    model.materials_coeff_A = Param() # Defines a parameter `materials_coeff_A` for the materials constraint coefficient of `xA`.
    model.materials_coeff_B = Param() # Defines a parameter `materials_coeff_B` for the materials constraint coefficient of `xB`.
    model.materials_rhs = Param() # Defines a parameter `materials_rhs` for the right-hand side of the materials constraint.
    model.labor_coeff_A = Param() # Defines a parameter `labor_coeff_A` for the labor constraint coefficient of `xA`.
    model.labor_coeff_B = Param() # Defines a parameter `labor_coeff_B` for the labor constraint coefficient of `xB`.
    model.labor_rhs = Param() # Defines a parameter `labor_rhs` for the right-hand side of the labor constraint.

    # Defines the objective function rule
    def obj_rule(model):
        return model.obj_coeff_A * model.xA + model.obj_coeff_B * model.xB # Specifies the objective function to maximize profit.
    model.obj = Objective(rule=obj_rule, sense=maximize) # Creates the objective function object with the defined rule and maximization sense.

    # Defines the materials constraint rule
    def materials_rule(model):
        return model.materials_coeff_A * model.xA + model.materials_coeff_B * model.xB <= model.materials_rhs # Specifies the materials constraint.
    model.materials = Constraint(rule=materials_rule) # Creates the materials constraint object with the defined rule.

    # Defines the labor constraint rule
    def labor_rule(model):
        return model.labor_coeff_A * model.xA + model.labor_coeff_B * model.xB <= model.labor_rhs # Specifies the labor constraint.
    model.labor = Constraint(rule=labor_rule) # Creates the labor constraint object with the defined rule.
    return model # Returns the built abstract model.

# Data for Example 2.2.1
data_221 = {
    None: {
        'obj_coeff_A': {None: 22}, 'obj_coeff_B': {None: 28}, # Objective function coefficients for xA and xB
        'materials_coeff_A': {None: 8}, 'materials_coeff_B': {None: 10}, 'materials_rhs': {None: 3400}, # Materials constraint coefficients and RHS
        'labor_coeff_A': {None: 2}, 'labor_coeff_B': {None: 3}, 'labor_rhs': {None: 960} # Labor constraint coefficients and RHS
    }
}
model_221 = build_abstract_model_221().create_instance(data_221) # Creates a concrete model instance by loading data into the abstract model.
solve_model(model_221, "Example 2.2.1") # Solves the model instance and prints the results.


 Solving Example 2.2.1 

Optimal Solution:
  xA = 150.0
  xB = 220.0

Objective Value: 9460.0


# Additional Examples

## Example 2.2.2: Satellite Launch Scheduling

### Problem Statement
A space agency wants to schedule satellite launches over three years. There are two types of payloads (T1 and T2), each with different success probabilities, costs, and preparation times. The agency wants to maximize its expected net income while meeting policy constraints and ensuring a minimum number of successful deployments.

### Model Formulation
1. **Decision Variables**:
   - Let \( $x_1$ \) = number of T1 payloads launched.
   - Let \( $x_2$ \) = number of T2 payloads launched.

2. **Objective Function**:
   - Maximize expected net income: \( $z$ = 0.225 $x_1$ + 0.2$ x_2$ \).

3. **Constraints**:
   - Policy constraints: \( $x_1$ $\leq$ 88 \), \( $x_2$ $\leq$ 126 \).
   - Successful deployments: \( 0.85 $x_1$ + 0.75 $x_2$ $\geq$ 60 \).
   - Preparation time: \( 2 $x_1$ + $x_2$ $\leq$ 156 \).
   - Non-negativity: \( $x_1$ $\geq$ 0 \), \( $x_2$ $\geq$ 0 \).

### Procedure
1. Define the decision variables \( $x_1$ \) and \( $x_2$ \).
2. Set up the objective function to maximize expected net income.
3. Add constraints for policy limits, successful deployments, and preparation time.
4. Solve the model using the CBC solver to determine the optimal number of payloads to launch.


### Code implementation

In [ ]:
# ========================================
# Example 2.2.2: Satellite launch scheduling
# ========================================

from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_222():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)  # T1 payloads launched
    model.x2 = Var(within=NonNegativeReals)  # T2 payloads launched

    # Parameters for objective coefficients and constraints
    model.obj_coeff = Param(['x1', 'x2'], within=Any)
    model.policy_constraints = Param([1, 2], within=Any)  # Maximum values for x1 and x2
    model.successful_deployments_coeffs = Param([1, 2], within=Any)  # Coefficients for successful deployments
    model.preparation_time_coeffs = Param([1, 2], within=Any)  # Coefficients for preparation time constraint

    # Objective function
    def obj_rule(model):
        return model.obj_coeff['x1'] * model.x1 + model.obj_coeff['x2'] * model.x2
    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Constraints
    # Policy constraints
    def policy_constraint1_rule(model):
        return model.x1 <= model.policy_constraints[1]
    model.policy_constraint1 = Constraint(rule=policy_constraint1_rule)

    def policy_constraint2_rule(model):
        return model.x2 <= model.policy_constraints[2]
    model.policy_constraint2 = Constraint(rule=policy_constraint2_rule)

    # Successful deployments constraint
    def successful_deployments_rule(model):
        return model.successful_deployments_coeffs[1] * model.x1 + model.successful_deployments_coeffs[2] * model.x2 >= 60
    model.successful_deployments = Constraint(rule=successful_deployments_rule)

    # Preparation time constraint
    def preparation_time_rule(model):
        return model.preparation_time_coeffs[1] * model.x1 + model.preparation_time_coeffs[2] * model.x2 <= 156
    model.preparation_time = Constraint(rule=preparation_time_rule)

    return model

data_222 = {
    None: {
        'obj_coeff': {'x1': 0.225, 'x2': 0.2},  # Objective function coefficients
        'policy_constraints': {1: 88, 2: 126},  # Maximum values for x1 and x2
        'successful_deployments_coeffs': {1: 0.85, 2: 0.75},  # Coefficients for successful deployments
        'preparation_time_coeffs': {1: 2, 2: 1},  # Coefficients for preparation time constraint
    }
}

# Instantiate the model
model_222 = build_abstract_model_222()
instance_222 = model_222.create_instance(data_222)
solve_model(instance_222, "Example 2.2.2")


 Solving Example 2.2.2 

Optimal Solution:
  x1 = 15.0
  x2 = 126.0

Objective Value: 28.575000000000003


## Example 2.2.3: Production and Inventory Management

### Problem Statement
A company wants to minimize its combined production and inventory costs over a four-week period. The company must meet weekly demand while maintaining a minimum inventory level and staying within production capacity limits.

### Model Formulation
1. **Decision Variables**:
   - Let \( $x_i$ \) = number of units produced in week \( $i$ \) (for \( $i$ = 1, 2, 3, 4 \)).
   - Let \( $A_i$ \) = inventory at the end of week \( $i$ \).

2. **Objective Function**:
   - Minimize total cost: \( $z$ = 15($x_1$ + $x_2$ + $x_3$ + $x_4$) + 3($A_1$ + $A_2$ + $A_3$) \).

3. **Constraints**:
   - Inventory balance: \( $A_1$ = 250 + $x_1$ - 900 \), \( $A_2$ = $A_1$ + $x_2$ - 600 \), etc.
   - Minimum inventory: \( $A_i$ $\geq$ 50 \) for \( $i$ = 1, 2, 3 \).
   - Production capacity: \( 500 $\leq$ $x_i$ $\leq$ $\text{capacity}$ \) for each week.
   - Non-negativity: \( $x_i$ $\geq$ 0 \), \( $A_i$ $\geq$ 0 \).

### Procedure
1. Define the decision variables \( $x_i$ \) and \( $A_i$ \).
2. Set up the objective function to minimize total cost.
3. Add constraints for inventory balance, minimum inventory, and production capacity.
4. Solve the model using the CBC solver to determine the optimal production and inventory levels.

### Code implementation

In [ ]:
# ========================================
# Example 2.2.3: Production and inventory cost minimization
# ========================================

from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_223():
    model = AbstractModel()
    model.T = Set()
    model.T_inv = Set()
    model.initial_inventory = Param()
    model.demand = Param(model.T)
    model.capacity = Param(model.T)
    model.production_cost = Param()
    model.inventory_cost = Param()
    model.min_production = Param()
    model.min_inventory = Param()

    model.x = Var(model.T, within=NonNegativeReals)
    model.A = Var(model.T_inv, within=NonNegativeReals)

    def obj_rule(model):
        return sum(model.production_cost * model.x[t] for t in model.T) + \
               sum(model.inventory_cost * model.A[t] for t in model.T_inv)
    model.obj = Objective(rule=obj_rule, sense=minimize)

    def inventory_balance_rule(model, t):
        if t == 1:
            return model.A[t] == model.initial_inventory + model.x[t] - model.demand[t]
        else:
            if t <= max(model.T_inv):
                return model.A[t-1] + model.x[t] - model.demand[t] == model.A[t]
            else:
                return model.A[t-1] + model.x[t] - model.demand[t] == 0
    model.inventory_balance = Constraint(model.T, rule=inventory_balance_rule)

    model.production_min = Constraint(model.T, rule=lambda m, t: m.x[t] >= m.min_production)
    model.production_max = Constraint(model.T, rule=lambda m, t: m.x[t] <= m.capacity[t])
    model.inventory_min = Constraint(model.T_inv, rule=lambda m, t: m.A[t] >= m.min_inventory)
    return model

data_223 = {None: {
    'T': {1, 2, 3, 4}, 'T_inv': {1, 2, 3},
    'initial_inventory': {None: 250},
    'demand': {1: 900, 2: 600, 3: 800, 4: 600},
    'capacity': {1: 800, 2: 700, 3: 600, 4: 800},
    'production_cost': {None: 15}, 'inventory_cost': {None: 3},
    'min_production': {None: 500}, 'min_inventory': {None: 50}
}}
model_223 = build_abstract_model_223().create_instance(data_223)
solve_model(model_223, "Example 2.2.3")


 Solving Example 2.2.3 

Optimal Solution:
  x[1] = 800.0
  x[2] = 700.0
  x[3] = 600.0
  x[4] = 550.0
  A[1] = 150.0
  A[2] = 250.0
  A[3] = 50.0

Objective Value: 41100.0


## Example 2.2.4: Freeze-Dried Vegetable Mixture

### Problem Statement
A company wants to minimize the cost of a freeze-dried vegetable mixture while meeting nutritional requirements and ingredient constraints. The mixture must contain specific amounts of iron, phosphorus, and calcium, and the proportions of beans and potatoes are limited.

### Model Formulation
1. **Decision Variables**:
   - Let \( $x_1$, $x_2$, $x_3$, $x_4$, $x_5$ \) = pounds of beans, corn, broccoli, cabbage, and potatoes, respectively.

2. **Objective Function**:
   - Minimize cost: \( $z$ = 20 $x_1$ + 18 $x_2$ + 32 $x_3$ + 28 $x_4$ + 16 $x_5$ \).

3. **Constraints**:
   - Ingredient limits: \( $x_1$ $\leq$ 0.4 ($x_1$ + $x_2$ + $x_3$ + $x_4$ + $x_5$) \), \( $x_5$ $\leq$ 0.32 ($x_1$ + $x_2$ + $x_3$ + $x_4$ + $x_5$) \).
   - Nutritional requirements: Iron, phosphorus, and calcium constraints.
      - \( 0.5 $x_1$ + 0.5 $x_2$ + 1.2 $x_3$ + 0.3 $x_4$ + 0.4 $x_5$ $\geq$ 5,000 \)
      - \( 10 $x_1$ + 20 $x_2$ + 40 $x_3$ + 30 $x_4$ + 50 $x_5$ $\geq$ 36,000 \)
      - \( 200 $x_1$ + 280 $x_2$ + 800 $x_3$ + 420 $x_4$ + 360 $x_5$ $\geq$ 28,000 \)
   - Non-negativity: \( $x_i$ $\geq$ 0 \) for \( 1 $\geq$ $i$ $\geq$ 5 \).

### Procedure
1. Define the decision variables \( $x_1$, $x_2$, $x_3$, $x_4$, $x_5$ \).
2. Set up the objective function to minimize cost.
3. Add constraints for ingredient limits and nutritional requirements.
4. Solve the model using the CBC solver to determine the optimal mixture.


### Code implementation

In [ ]:
# ========================================
# Example 2.2.4: Minimizing cost of vegetable mixture
# ========================================

from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_224():
    model = AbstractModel()

    # Decision variables for pounds of beans, corn, broccoli, cabbage, and potatoes
    model.x1 = Var(within=NonNegativeReals)
    model.x2 = Var(within=NonNegativeReals)
    model.x3 = Var(within=NonNegativeReals)
    model.x4 = Var(within=NonNegativeReals)
    model.x5 = Var(within=NonNegativeReals)

    # Parameters
    model.total_ingredients = Param(within=NonNegativeReals, default=1)  # Total of all ingredients
    model.iron_requirement = Param(within=NonNegativeReals, default=5000)
    model.phosphorus_requirement = Param(within=NonNegativeReals, default=36000)
    model.calcium_requirement = Param(within=NonNegativeReals, default=28000)

    # Objective function: Minimize cost
    def obj_rule(model):
        return 20*model.x1 + 18*model.x2 + 32*model.x3 + 28*model.x4 + 16*model.x5
    model.obj = Objective(rule=obj_rule, sense=minimize)

    # Constraints

    # Ingredient limits
    def ingredient_limit_1_rule(model):
        return model.x1 <= 0.4 * (model.x1 + model.x2 + model.x3 + model.x4 + model.x5)
    model.ingredient_limit_1 = Constraint(rule=ingredient_limit_1_rule)

    def ingredient_limit_2_rule(model):
        return model.x5 <= 0.32 * (model.x1 + model.x2 + model.x3 + model.x4 + model.x5)
    model.ingredient_limit_2 = Constraint(rule=ingredient_limit_2_rule)

    # Nutritional requirements
    def iron_rule(model):
        return 0.5 * model.x1 + 0.5 * model.x2 + 1.2 * model.x3 + 0.3 * model.x4 + 0.4 * model.x5 >= model.iron_requirement
    model.iron_constraint = Constraint(rule=iron_rule)

    def phosphorus_rule(model):
        return 10 * model.x1 + 20 * model.x2 + 40 * model.x3 + 30 * model.x4 + 50 * model.x5 >= model.phosphorus_requirement
    model.phosphorus_constraint = Constraint(rule=phosphorus_rule)

    def calcium_rule(model):
        return 200 * model.x1 + 280 * model.x2 + 800 * model.x3 + 420 * model.x4 + 360 * model.x5 >= model.calcium_requirement
    model.calcium_constraint = Constraint(rule=calcium_rule)

    return model

data_224 = {
    None: {
        'total_ingredients': {None: 1},  # Mapping required for each parameter
        'iron_requirement': {None: 5000},
        'phosphorus_requirement': {None: 36000},
        'calcium_requirement': {None: 28000}
    }
}

# Instantiate the model
model_224 = build_abstract_model_224()
instance_224 = model_224.create_instance(data_224)
solve_model(instance_224, "Example 2.2.4")


 Solving Example 2.2.4 

Optimal Solution:
  x1 = 0.0
  x2 = 0.0
  x3 = 4166.6667
  x4 = 0.0
  x5 = 0.0

Objective Value: 133333.3344


## Example 2.2.5: Sawmill Profit Maximization

### Problem Statement
A sawmill produces fir and spruce logs, each with different processing times and by-products. The sawmill wants to maximize profits while considering the utilization of by-products (sawdust) and market constraints.

### Model Formulation
1. **Decision Variables**:
   - Let \( $x_1$ \) = number of fir logs produced.
   - Let \( $x_2$ \) = number of spruce logs produced.
   - Let \( $x_3$ \) = pounds of spruce sawdust used.
   - Let \( $x_4$ \) = pounds of spruce sawdust destroyed.

2. **Objective Function**:
   - Maximize profit: \( $z$ = 4.38 $x_1$ + 5 $x_2$ + 0.66 $x_3$ - 0.15 $x_4$ \).

3. **Constraints**:
   - Sawdust balance: \( 5 $x_2$ = $x_3$ + $x_4$ \).
   - Market limit for pressed wood: \( $\frac{3}{5} $$x_3$ $\leq$ 60 \).
   - Machine time constraints: Bark peeler, slab saw, and planer.
      - \( 3 $x_1$ + 2 $x_2$ $\leq$ 10 \)
      - \( 5 $x_1$ + 6 $x_2$ $\leq$ 12 \)
      - \( 2 $x_1$ + 2 $x_2$ $\leq$ 14 \)
   - Non-negativity: \( $x_1$, $x_2$, $x_3$, $x_4$ $\geq$ 0 \).

### Procedure
1. Define the decision variables \( $x_1$, $x_2$, $x_3$, $x_4$ \).
2. Set up the objective function to maximize profit.
3. Add constraints for sawdust balance, market limits, and machine time.
4. Solve the model using the CBC solver to determine the optimal production levels.

### Code implementation

In [ ]:
# ========================================
# Example 2.2.5: Maximizing profit in a sawmill
# ========================================

from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_225():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)  # Fir logs produced
    model.x2 = Var(within=NonNegativeReals)  # Spruce logs produced
    model.x3 = Var(within=NonNegativeReals)  # Pounds of spruce sawdust used
    model.x4 = Var(within=NonNegativeReals)  # Pounds of spruce sawdust destroyed

    # Parameters for objective coefficients and constraints
    model.obj_coeff = Param(['x1', 'x2', 'x3', 'x4'], within=Any)
    model.machine_time_constraints = Param([1, 2, 3], within=Any)  # For machine time constraints
    model.sawdust_balance_coeffs = Param([1, 2], within=Any)  # Sawdust balance coefficients

    # Objective function
    def obj_rule(model):
        return model.obj_coeff['x1'] * model.x1 + model.obj_coeff['x2'] * model.x2 + \
               model.obj_coeff['x3'] * model.x3 + model.obj_coeff['x4'] * model.x4
    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Constraints for sawdust balance
    def sawdust_balance_rule(model):
        return model.sawdust_balance_coeffs[1] * model.x2 == model.x3 + model.x4
    model.sawdust_balance = Constraint(rule=sawdust_balance_rule)

    # Market limit for pressed wood
    def market_limit_rule(model):
        return (3 / 5) * model.x3 <= 60
    model.market_limit = Constraint(rule=market_limit_rule)

    # Machine time constraints
    def machine_time1_rule(model):
        return 3 * model.x1 + 2 * model.x2 <= model.machine_time_constraints[1]
    model.machine_time1 = Constraint(rule=machine_time1_rule)

    def machine_time2_rule(model):
        return 5 * model.x1 + 6 * model.x2 <= model.machine_time_constraints[2]
    model.machine_time2 = Constraint(rule=machine_time2_rule)

    def machine_time3_rule(model):
        return 2 * model.x1 + 2 * model.x2 <= model.machine_time_constraints[3]
    model.machine_time3 = Constraint(rule=machine_time3_rule)

    return model

data_225 = {
    None: {
        'obj_coeff': {'x1': 4.38, 'x2': 5, 'x3': 0.66, 'x4': -0.15},  # Objective function coefficients
        'machine_time_constraints': {1: 10, 2: 12, 3: 14},  # Machine time limits for the constraints
        'sawdust_balance_coeffs': {1: 5, 2: 1},  # Sawdust balance coefficients
    }
}

# Instantiate the model
model_225 = build_abstract_model_225()
instance_225 = model_225.create_instance(data_225)
solve_model(instance_225, "Example 2.2.5")


 Solving Example 2.2.5 

Optimal Solution:
  x1 = 0.0
  x2 = 2.0
  x3 = 10.0
  x4 = 0.0

Objective Value: 16.6


## Example 2.2.6: Dual Processor Job Scheduling

### Problem Statement
A computing facility must schedule administrative and scientific jobs on two processors. The goal is to minimize the total time the system is occupied while ensuring each processor is used for at least 10 hours.

### Model Formulation
1. **Decision Variables**:
   - Let \( $x_1$ \) = number of administrative jobs.
   - Let \( $x_2$ \) = number of scientific jobs.
   - Let \( $x_3$ \) = maximum time spent on either processor.

2. **Objective Function**:
   - Minimize \( $x_3$ \), where \( $x_3$ = $\max$ (2 $x_1$ + 5 $x_2$, 6 $x_1$ + 3 $x_2$) \).

3. **Constraints**:
   - Processor time: \( 2 $x_1$ + 5 $x_2$ $\geq$ 36,000 \), \( 6 $x_1$ + 3 $x_2$ $\geq$ 36,000 \).
   - Linearization : \( $x_3$ $\geq$ 2 $x_1$ + 5 $x_2$\) and \( $x_3$ $\geq$ 6 $x_1$ + 3 $x_2$ \)
   - Non-negativity: \( $x_1$, $x_2$, $x_3$ $\geq$ 0 \).

### Procedure
1. Define the decision variables \( $x_1$, $x_2$, $x_3$ \).
2. Set up the objective function to minimize the maximum processor time.
3. Add constraints for processor time requirements.
4. Solve the model using the CBC solver to determine the optimal job schedule.

### Code implementation

In [ ]:
# ========================================
# Example 2.2.6: Minimizing computing time on dual processors
# ========================================

from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_226():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)
    model.x2 = Var(within=NonNegativeReals)
    model.x3 = Var(within=NonNegativeReals)  # The maximum time variable

    # Parameters for processor coefficients (indexed by 1 and 2 for x1 and x2)
    model.processor1_coeffs = Param([1, 2], within=Any)
    model.processor2_coeffs = Param([1, 2], within=Any)
    model.processor1_min = Param(within=Any)
    model.processor2_min = Param(within=Any)

    # Constraints for processor time (ensuring both expressions are >= 36,000)
    def processor1_constraint_rule(model):
        return model.processor1_coeffs[1] * model.x1 + model.processor1_coeffs[2] * model.x2 >= model.processor1_min
    model.processor1_constraint = Constraint(rule=processor1_constraint_rule)

    def processor2_constraint_rule(model):
        return model.processor2_coeffs[1] * model.x1 + model.processor2_coeffs[2] * model.x2 >= model.processor2_min
    model.processor2_constraint = Constraint(rule=processor2_constraint_rule)

    # Linearization for the max function: x3 should be greater than both expressions
    def linearization_constraint1_rule(model):
        return model.x3 >= model.processor1_coeffs[1] * model.x1 + model.processor1_coeffs[2] * model.x2
    model.linearization_constraint1 = Constraint(rule=linearization_constraint1_rule)

    def linearization_constraint2_rule(model):
        return model.x3 >= model.processor2_coeffs[1] * model.x1 + model.processor2_coeffs[2] * model.x2
    model.linearization_constraint2 = Constraint(rule=linearization_constraint2_rule)

    # Objective function: Minimize x3
    def obj_rule(model):
        return model.x3  # Minimize the maximum time
    model.obj = Objective(rule=obj_rule)

    return model

data_226 = {None: {
    'processor1_coeffs': {1: 1.0, 2: 2.0}, 'processor1_min': {None: 36000},
    'processor2_coeffs': {1: 2.0, 2: 1.0}, 'processor2_min': {None: 18000}
}}
model_226 = build_abstract_model_226()
instance_226 = model_226.create_instance(data_226)
solve_model(instance_226, "Example 2.2.6")


 Solving Example 2.2.6 

Optimal Solution:
  x1 = 0.0
  x2 = 18000.0
  x3 = 36000.0

Objective Value: 36000.0


# End